# Wikipedia爬蟲練習
## 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

In [14]:
import requests
import re
from bs4 import BeautifulSoup

### 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [15]:
input_keyword = "愛的迫降"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

愛的迫降: %E6%84%9B%E7%9A%84%E8%BF%AB%E9%99%8D
/wiki/%E6%84%9B%E7%9A%84%E8%BF%AB%E9%99%8D


### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [16]:
# 模擬封包的標頭
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text())



2019年12月14日 (2019-12-14)－2020年2月16日 (2020-02-16)

《愛的迫降》（韓語：사랑의 불시착），為韓國tvN於2019年12月14日起播出的週末連續劇，由《火星生活》、《羅曼史是別冊附錄》的李政孝導演與《來自星星的你》、《製作人》、《藍色海洋的傳說》的朴智恩作家合作打造。此劇講述某天因為狂風而發生滑翔傘事故，被迫在北朝鮮降落的南韓財閥繼承人尹世理（孫藝真飾），與隱藏、守護她並愛上她的軍官李正赫（炫彬飾）之間的愛情故事。[1][2][3][4][5][6]

Netflix於12月14日起每週六日22:00 (UTC+8)全球同步上線。

受不了良心的譴責，於是決定幫助正赫且告知武赫被害的真相。



週五六時段劇集

週六時段劇集

週末時段劇集



### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [17]:
for ext_link in content:
    a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

外部連結: [韓國] /wiki/%E9%9F%93%E5%9C%8B
外部連結: [tvN] /wiki/TvN
外部連結: [週末連續劇] /wiki/TvN%E9%80%B1%E6%9C%AB%E9%80%A3%E7%BA%8C%E5%8A%87
外部連結: [火星生活] /wiki/%E7%81%AB%E6%98%9F%E7%94%9F%E6%B4%BB_(%E9%9F%93%E5%9C%8B%E9%9B%BB%E8%A6%96%E5%8A%87)
外部連結: [羅曼史是別冊附錄] /wiki/%E7%BE%85%E6%9B%BC%E5%8F%B2%E6%98%AF%E5%88%A5%E5%86%8A%E9%99%84%E9%8C%84
外部連結: [來自星星的你] /wiki/%E4%BE%86%E8%87%AA%E6%98%9F%E6%98%9F%E7%9A%84%E4%BD%A0
外部連結: [製作人] /wiki/%E8%A3%BD%E4%BD%9C%E4%BA%BA_(%E9%9F%93%E5%9C%8B%E9%9B%BB%E8%A6%96%E5%8A%87)
外部連結: [藍色海洋的傳說] /wiki/%E8%97%8D%E8%89%B2%E6%B5%B7%E6%B4%8B%E7%9A%84%E5%82%B3%E8%AA%AA
外部連結: [朴智恩] /wiki/%E6%9C%B4%E6%99%BA%E6%81%A9_(%E7%B7%A8%E5%8A%87)
外部連結: [孫藝真] /wiki/%E5%AD%AB%E8%97%9D%E7%9C%9F
外部連結: [炫彬] /wiki/%E7%82%AB%E5%BD%AC
外部連結: [Netflix] /wiki/Netflix
外部連結: [UTC+8] /wiki/UTC%2B8


## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [21]:
def WikiArticle(key_word_link, key_word, recursive):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        '''
        
        Your code here
        
        '''
       
        for paragraph in content:
            file_name = key_word.replace('/', '_')
            f  = open('WiKiArticle/' + file_name, 'a+', encoding = 'utf-8')
            f.write( str(paragraph.get_text()) + '\n')
            f.close
        
        
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})
        '''
        
        Your code here
        
        '''
        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
            if len(a_tag) > 0:
                for link_string in a_tag:
                    a_link = link_string['href']
                    a_keyword = link_string.get_text()
                    external_link_dict[a_link] = a_keyword
                    
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():
                WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程
                

/wiki/%E6%84%9B%E7%9A%84%E8%BF%AB%E9%99%8D


### 執行前個步驟定義好的爬蟲主程式

In [22]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 1

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] - /wiki/%E6%84%9B%E7%9A%84%E8%BF%AB%E9%99%8D (愛的迫降)
遞迴層[1] - /wiki/%E9%9F%93%E5%9C%8B (韓國)
遞迴層[1] - /wiki/TvN (tvN)
遞迴層[1] - /wiki/TvN%E9%80%B1%E6%9C%AB%E9%80%A3%E7%BA%8C%E5%8A%87 (週末連續劇)
遞迴層[1] - /wiki/%E7%81%AB%E6%98%9F%E7%94%9F%E6%B4%BB_(%E9%9F%93%E5%9C%8B%E9%9B%BB%E8%A6%96%E5%8A%87) (火星生活)
遞迴層[1] - /wiki/%E7%BE%85%E6%9B%BC%E5%8F%B2%E6%98%AF%E5%88%A5%E5%86%8A%E9%99%84%E9%8C%84 (羅曼史是別冊附錄)
遞迴層[1] - /wiki/%E4%BE%86%E8%87%AA%E6%98%9F%E6%98%9F%E7%9A%84%E4%BD%A0 (來自星星的你)
遞迴層[1] - /wiki/%E8%A3%BD%E4%BD%9C%E4%BA%BA_(%E9%9F%93%E5%9C%8B%E9%9B%BB%E8%A6%96%E5%8A%87) (製作人)
遞迴層[1] - /wiki/%E8%97%8D%E8%89%B2%E6%B5%B7%E6%B4%8B%E7%9A%84%E5%82%B3%E8%AA%AA (藍色海洋的傳說)
遞迴層[1] - /wiki/%E6%9C%B4%E6%99%BA%E6%81%A9_(%E7%B7%A8%E5%8A%87) (朴智恩)
遞迴層[1] - /wiki/%E5%AD%AB%E8%97%9D%E7%9C%9F (孫藝真)
遞迴層[1] - /wiki/%E7%82%AB%E5%BD%AC (炫彬)
遞迴層[1] - /wiki/Netflix (Netflix)
遞迴層[1] - /wiki/UTC%2B8 (UTC+8)
